In [0]:
######### image augmentaion #########
#data augmentation using python and keras
import keras 
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
    rotation_range = -40,
    width_shift_range = 0.2,
    height_shift_range = 0.2, 
    shear_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True,
    fill_mode = "nearest")

img = load_img("./sample.jpg")
x = img_to_array(img)
x = x.reshape((1, ) + x.shape)  
#this is an numpy array of shape(1, 3, 150, 150)


#.flow funtion generates batches of randomly transformed images
# and saves the results to the preview directory
#batch_size ===> noumber if images processing at a time
i = 0
for batch in datagen.flow(x, batch_size = 1,
                          save_to_dir = "preview", save_prefix = "cat", 
                          save_format = "jpeg"):
  i += 1
  if i> 20:
    break




In [0]:
#keras-tuner is an optimzier 
!pip install keras-tuner

     |████████████████████████████████| 61kB 6.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=877cf8f9c82a1a8ca7549027323dd2870274adbe2552455e8c2d2c27ac027302
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=91f91718b28f605ebef5dbbe66466464b19961527c803bcc4b37d7efaaedf4ac
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization



In [0]:
# fashion mnist data set
fashion_mnist = keras.datasets.fashion_mnist

In [0]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
#scaling down the images between 0 and 1
train_images = train_images / 255
test_images = test_images/255
 

In [0]:
 train_images = train_images.reshape(len(train_images), 28, 28, 1)
 test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [0]:
# def build_model(hp):
#   model = tf.keras.Sequential()
#   #hp.Int will craete filters a range of integer between min_valuea and 
#   #max_value
#   model.add(Conv2D(
#       #filters is the number of output filters in the convolution
#       filters =  hp.Int("conv_1_filter", min_value = 32, max_value = 128, step = 16), 
#       #kernel_size = filter_size         
#       kernel_size = hp.Choice("conv_1_kernel", values = [3, 5]),
#       activation = "relu", input_shape = (28, 28, 1)))
#   model.add(Conv2D(
#       filters =  hp.Int("conv_1_filter", min_value = 32, max_value = 128, step = 16), 
#       kernel_size = hp.Choice("conv_1_kernel", values = [3, 5]),
#       activation = "relu"))
#   model.add(Flatten())
#   model.add(Dense(
#       units = hp.Int("dense_1_units", min_value = 32, max_value = 128), 
#       activation = "relu"))
#   model.add(Dense(10, activation = "softmax"))


#   model.compile(optimizer = keras.optimizers.Adam(
#       hp.Choice('learning_rate', values = [ 0.01, 0.001])), 
#       loss = "categorical_crossentropy", metrics = ["accuracy"])
  

#   return model

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters



In [0]:
tuner_search = RandomSearch(build_model, objective = "val_accuracy", 
                            max_trials = 5, directory = "output",
                            project_name = "mnist_fashion")

In [0]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)
#validation_split ==> training data will be split on that ratio fror validation 

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4178 - accuracy: 0.8496 - val_loss: 0.3168 - val_accuracy: 0.8888
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2694 - accuracy: 0.9017 - val_loss: 0.2793 - val_accuracy: 0.9010
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2145 - accuracy: 0.9205 - val_loss: 0.2577 - val_accuracy: 0.9078


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5170 - accuracy: 0.8134 - val_loss: 0.4208 - val_accuracy: 0.8510
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3814 - accuracy: 0.8590 - val_loss: 0.3664 - val_accuracy: 0.8648
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3433 - accuracy: 0.8724 - val_loss: 0.4265 - val_accuracy: 0.8503


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3929 - accuracy: 0.8599 - val_loss: 0.2860 - val_accuracy: 0.8915
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2505 - accuracy: 0.9074 - val_loss: 0.2634 - val_accuracy: 0.9063
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1910 - accuracy: 0.9282 - val_loss: 0.2455 - val_accuracy: 0.9148


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4038 - accuracy: 0.8529 - val_loss: 0.3113 - val_accuracy: 0.8877
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2644 - accuracy: 0.9020 - val_loss: 0.2622 - val_accuracy: 0.9070
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2142 - accuracy: 0.9208 - val_loss: 0.2522 - val_accuracy: 0.9087


Epoch 1/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.4058 - accuracy: 0.8534 - val_loss: 0.3226 - val_accuracy: 0.8793
Epoch 2/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2675 - accuracy: 0.9006 - val_loss: 0.2656 - val_accuracy: 0.9053
Epoch 3/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2121 - accuracy: 0.9204 - val_loss: 0.2778 - val_accuracy: 0.8987


INFO:tensorflow:Oracle triggered exit


In [0]:
model = tuner_search.get_best_models(num_models = 1)[0]


In [0]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        57648     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1858640   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,917,578
Trainable params: 1,917,578
Non-trainable params: 0
_________________________________________________________________


In [0]:
 #we are training the model with the training 
 model.fit(train_images, train_labels, epochs = 10, 
           validation_split = 0.1, initial_epoch = 3)  
 #initial_epoch == is the same epoch with which we tubned the model 
 #because we have already done first 3 epochs  

Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1428 - accuracy: 0.9460 - val_loss: 0.2841 - val_accuracy: 0.9117
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1039 - accuracy: 0.9613 - val_loss: 0.2963 - val_accuracy: 0.9170
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0756 - accuracy: 0.9722 - val_loss: 0.2909 - val_accuracy: 0.9150
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0559 - accuracy: 0.9797 - val_loss: 0.3570 - val_accuracy: 0.9140
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0438 - accuracy: 0.9843 - val_loss: 0.3681 - val_accuracy: 0.9182
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0334 - accuracy: 0.9877 - val_loss: 0.4140 - val_accuracy: 0.9058
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0293 - accuracy: 0.9896 - val_loss: 0.4604 - val_a

In [0]:
#converting an arr to image 
import numpy as np
array_to_img(np.expand_dims(train_images[0], axis=2))

In [0]:
#predicting a single image
model.predict_classes(train_images[0:1])

NameError: ignored

(28, 28)